In [1]:
!pip install telethon


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.2/702.2 kB 10.7 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=114b6a776d8da51b813f4763c7911830211e4ed102ab6239757e31705a1bdfc1
  Stored in directory: /root/.cache/pip/wheels/d6/84/5f/ea6aef85a93c7e1922486369874f4740a5642d261e09c59140
Successfully built pyaes


In [3]:
from telethon import TelegramClient, sync
import pandas as pd


In [4]:
api_id = '21237365'  # Replace with your API ID
api_hash = '92686857a114d7bf8be29a2ff99b7b92'  # Replace with your API Hash

# Create and start the client
client = TelegramClient('session_name', api_id, api_hash)
await client.start()


Please enter your phone (or bot token): +918798066530
Please enter the code you received: 57939
Signed in successfully as MoMataKiAshirwaad; remember to not break the ToS or you will risk an account ban!


In [5]:
#to put rate limit
from telethon.errors import FloodWaitError
import asyncio
import nest_asyncio
from telethon import TelegramClient, errors

In [7]:

# Apply nest_asyncio to allow nested event loops in environments like Jupyter/Colab
nest_asyncio.apply()

async def scrape_channel(channel_username, limit=100):
    messages = []
    try:
        async for message in client.iter_messages(channel_username, limit=limit):
            if message.text:  # Ensure message text is not None
                messages.append(message.text)
    except errors.FloodWaitError as e:
        print(f"Rate limit hit, waiting for {e.seconds} seconds...")
        await asyncio.sleep(e.seconds)  # Pause for the required duration
        return await scrape_channel(channel_username, limit)  # Retry after waiting
    return messages

# Connect to the client and scrape the channel
async def main(channel_username):
    await client.start()
    data = await scrape_channel(channel_username, limit=100)
    await client.disconnect()
    return data

# Run the main function
channel_username = 'LIQUIDEOFFICIAL'  # Replace with the channel's username
data = asyncio.run(main(channel_username))

# Print or process the data as needed
print(data)


['**NIFTY50 LEVELS FOR INTRADAY\n\nNifty is trading around 24120.\n\nRESISTANCE: Important resistance levels for nifty are 24180 and 24250.\n\nSUPPORT: Important support levels for nifty are 24075 and 24025**', 'Good Morning Telegram Family!\n\n**Market Cues for Today**\n\n📌 **Nifty**: The index has formed a bullish Harami candlestick pattern on the daily chart.\n\n- Support: 24,070\n- Resistance: 24,200 \n\n📌 **Bank Nifty**: The index has formed a small-bodied bullish candlestick pattern, with a minor upper shadow and a long lower shadow on the daily chart.\n\n- Support: 51,800\n- Resistance: 52,250 \n \n📌 **Long Build-Up in Stocks**:\n\n- Indraprastha Gas\n- \u2060Divi’s Laboratories \n- \u2060Mahanagar Gas\n- \u2060GNFC\n- \u2060Petronet LNG\n\n📌 **Short Build-Up in Stocks**: \n\n- Apollo Hospitals\n- \u2060Chola Finance\n- \u2060Godrej Properties\n- \u2060RBL Bank\n- \u2060Hero Motocorp\n\n📌 **Stocks under F&O Ban**: Nil', "📊 **Wild Swings on D-Street!**\n\nMarkets had another vola

In [80]:
import nest_asyncio
import asyncio
from telethon import TelegramClient, errors
import re
import pandas as pd
from datetime import datetime

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Define stock market indices to search for
stock_indices = [
    "Nifty", "Nifty50", "BankNifty", "FinNifty", "Sensex", "MidcapNifty"
]

# Regular expressions to identify support, resistance levels, and current trading value
support_pattern = re.compile(r"(support|support at)[^\d]*(\d+\.?\d*)\s*and[^\d]*(\d+\.?\d*)", re.IGNORECASE)
resistance_pattern = re.compile(r"(resistance|resistance at)[^\d]*(\d+\.?\d*)\s*and[^\d]*(\d+\.?\d*)", re.IGNORECASE)
current_trading_pattern = re.compile(r"(\bNifty\b|\bBankNifty\b|\bFinNifty\b|\bSensex\b|\bMidcapNifty\b)[^\d]*(\d+\.?\d*)", re.IGNORECASE)

# Regular expression to identify candlestick patterns
pattern_candlestick = re.compile(r"\b(bearish|bullish|harami|doji|hammer|shooting star|engulfing|spinning top)\b", re.IGNORECASE)

# Regular expressions to identify long buildup, short buildup, and F&O ban
long_buildup_pattern = re.compile(r"(long buildup|long build-up|long positions)", re.IGNORECASE)
short_buildup_pattern = re.compile(r"(short buildup|short build-up|short positions)", re.IGNORECASE)
fo_ban_pattern = re.compile(r"(F&O ban|under F&O ban)", re.IGNORECASE)

def find_stock_indices(message_text):
    """Search for stock market indices in a message."""
    indices_found = []
    for index in stock_indices:
        if re.search(r'\b' + re.escape(index) + r'\b', message_text, re.IGNORECASE):
            indices_found.append(index)
    return indices_found

def extract_levels(message_text):
    """Extract support and resistance levels from a message."""
    support_1, support_2, resistance_1, resistance_2 = None, None, None, None

    support_match = support_pattern.search(message_text)
    resistance_match = resistance_pattern.search(message_text)

    if support_match:
        support_1 = float(support_match.group(2))  # Extracted Support 1 level
        support_2 = float(support_match.group(3))  # Extracted Support 2 level
    if resistance_match:
        resistance_1 = float(resistance_match.group(2))  # Extracted Resistance 1 level
        resistance_2 = float(resistance_match.group(3))  # Extracted Resistance 2 level

    return support_1, support_2, resistance_1, resistance_2

def extract_current_trading_value(message_text):
    """Extract current trading value for a market index."""
    current_value = None
    match = current_trading_pattern.search(message_text)
    if match:
        current_value = float(match.group(2))  # Extracted current trading value
    return current_value

def extract_pattern(message_text):
    """Extract candlestick pattern from the message."""
    match = pattern_candlestick.search(message_text)
    return match.group(1).capitalize() if match else None

def extract_buildups(message_text):
    """Extract long buildup, short buildup, and F&O ban status from a message."""
    long_buildup = bool(long_buildup_pattern.search(message_text))
    short_buildup = bool(short_buildup_pattern.search(message_text))
    fo_ban = bool(fo_ban_pattern.search(message_text))
    return long_buildup, short_buildup, fo_ban

def extract_date(message):
    """Extract date from the message metadata (if available)."""
    return message.date.strftime('%Y-%m-%d %H:%M:%S')

async def scrape_channel(channel_username, limit=100):
    messages = []
    try:
        async for message in client.iter_messages(channel_username, limit=limit):
            if message.text:  # Ensure message text is not None
                indices_in_message = find_stock_indices(message.text)
                if indices_in_message:  # If the message contains stock indices
                    date = extract_date(message)
                    support_1, support_2, resistance_1, resistance_2 = extract_levels(message.text)
                    current_value = extract_current_trading_value(message.text)
                    pattern = extract_pattern(message.text)
                    long_buildup, short_buildup, fo_ban = extract_buildups(message.text)
                    for index in indices_in_message:
                        messages.append([date, index, support_1, support_2, resistance_1, resistance_2, current_value, pattern, long_buildup, short_buildup, fo_ban, message.text])  # Store all info
    except errors.FloodWaitError as e:
        print(f"Rate limit hit, waiting for {e.seconds} seconds...")
        await asyncio.sleep(e.seconds)  # Pause for the required duration
        return await scrape_channel(channel_username, limit)  # Retry after waiting
    return messages

# Connect to the client and scrape the channel
async def main(channel_username):
    await client.start()
    data = await scrape_channel(channel_username, limit=100)
    await client.disconnect()
    return data

# Run the main function
channel_username = 'LIQUIDEOFFICIAL'  # Replace with the channel's username
data = asyncio.run(main(channel_username))

# Convert the scraped data into a DataFrame
columns = ['Date', 'Stock Index', 'Support 1', 'Support 2', 'Resistance 1', 'Resistance 2',
           'Current', 'Pattern', 'Long Buildup', 'Short Buildup', 'Under F&O Ban', 'Message']
df = pd.DataFrame(data, columns=columns)

# Remove the unwanted columns: Long Buildup, Short Buildup, Under F&O Ban, Message
df = df.drop(columns=['Long Buildup', 'Short Buildup', 'Under F&O Ban', 'Message'])

# Display the DataFrame
print(df)

# Optionally, save the filtered data to a CSV file
#df.to_csv('stock_indices_with_support_resistance_current_pattern.csv', index=False)


                   Date Stock Index  Support 1  Support 2  Resistance 1  Resistance 2  Current  \
0   2024-12-02 05:38:26       Nifty    24075.0    24025.0       24180.0       24250.0  24120.0   
1   2024-12-02 05:38:26     Nifty50    24075.0    24025.0       24180.0       24250.0  24120.0   
2   2024-12-02 02:58:56       Nifty        NaN        NaN           NaN           NaN     24.0   
3   2024-12-01 13:52:31       Nifty        NaN        NaN           NaN           NaN   2024.0   
4   2024-11-29 05:32:18      Sensex    79400.0    79200.0       79900.0       80150.0  79670.0   
5   2024-11-29 04:47:55       Nifty    23885.0    23800.0       24065.0       24130.0  24000.0   
6   2024-11-29 04:47:55     Nifty50    23885.0    23800.0       24065.0       24130.0  24000.0   
7   2024-11-29 03:02:18       Nifty        NaN        NaN           NaN           NaN     23.0   
8   2024-11-28 06:19:37       Nifty    52000.0    51770.0       52300.0       52500.0  52180.0   
9   2024-11-28 06:19

In [82]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Save the filtered data to Google Drive as CSV
csv_filename = '/content/drive/MyDrive/intern5_stock_data.csv'
df.to_csv(csv_filename, index=False)

print(f'File saved to Google Drive: {csv_filename}')

Mounted at /content/drive
File saved to Google Drive: /content/drive/MyDrive/intern5_stock_data.csv


In [30]:
df = pd.DataFrame(data, columns=['message'])

In [31]:
import re

# Remove unwanted characters and links
df['message'] = df['message'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+|[^a-zA-Z0-9\s]', '', x))
df = df.dropna().reset_index(drop=True)


In [32]:
df = df.dropna()

In [33]:
from google.colab import drive
drive.mount('/content/drive')
df.to_csv('/content/drive/MyDrive/intern2_stock_data.csv', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
